In [29]:
import numpy as np
from scipy.io import loadmat
import os
import glob
import re
import sys
from six.moves import cPickle as pickle

print("import done")

import done


In [67]:
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]
    
def dataset_patient(folder_pat):
    #seizure preictal data
    files = sorted(glob.glob(folder_pat + "/*1.mat"), key=natural_key)
    holder_data = np.ndarray(shape=(len(files)/4, 400*10*60, 16), dtype=np.float32)
    holder_sequence = np.ndarray(shape=(len(files)), dtype=np.float32)
    
    sequence = 0
    for idx, fl in enumerate(files[:len(files)/4]):
        #get sequence from 'patient_sequence_result.mat'
        sequence = os.path.basename(fl)[:-4].split("_")[1]
        #open mat file
        mat = loadmat(fl)
        #convert from ndarray object to ndarray
        names = mat['dataStruct'].dtype.names
        ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
        #detect if columns out of order
        if not all(x<y for x, y in zip(ndata['channelIndices'][0], ndata['channelIndices'][0][1:])):
            print('WARNING: Columns out of order in file' + fl)
        
        holder_data[idx, :, :] = ndata['data']
        holder_sequence[idx] = ndata['sequence']
      
    print('pickling...')
    try:
        with open('train1.pickle', 'wb') as f:
            pickle.dump(holder_data, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to:', e)
    
    with open('train1.pickle', 'rb') as f:
        save = pickle.load(f)
        print(save[0:5,0:2,0:2])
    
dataset_patient("./train_1")

print('done')

[[[ 19.37265396  -1.63778603]
  [ 23.37265396  -4.63778591]]

 [[-36.38835144   5.95662212]
  [-33.38835144  -1.04337788]]

 [[ 21.97092438  12.75170517]
  [ 28.97092438   4.75170565]]

 [[  8.58492279  11.79246521]
  [ 12.58492279   3.79246545]]

 [[  0.54836392   4.35475588]
  [ -1.45163608   7.35475588]]]
done
